# Cosmicflows-4 Distance-Velocity API

This notebook explains how you can simply interact with Cosmicflows calculators within your *Python* code.

**Note:** This API can be also queried in any programming language of choice.

- Communication method: POST, GET
- Input data: JSON 

**CF4 DV-calculator API:** http://edd.ifa.hawaii.edu/CF4calculator/api.php

- Processing large lists of galaxies provided in JSON format. Below, `CF4_list` facilitates the data preparation for API calls. Refer to [Example 3](#example3) in this notebook for further details. The maximum number of galaxies per query is limited to 500.

## Python Example


### Simply copy the following functions into your code

The following code snippets provide interface functions to query each of the calculators.

Required packages:

- requests (to handle the online API requests)
- json (to translate between Python dictionaries and json format)
    

In [1]:
import requests
import json

def CF4(alpha, delta, system='supergalactic', 
                 parameter='distance', value=20):
    
    """
    Inputs: 
        alpha: (float) [deg]
            first coordinate parameter  (RA,  Glon, SGL)
        delta: (float) [deg]
            second coordinate parameter (Dec, Glat, SGB)  
        system: (string)
            coordinate system: 
            Options are:
                "equatorial"
                "galactic"
                "supergalactic"
        parameter: (string)
            the quantity whose value is provided
            Options are:
                "distance"
                "velocity"
        value: (float)
            the value of the input quantity
            distance in [Mpc] and velocity in [km/s]
        
    Output:
        A python dictionary which contains the distance and velocity of the 
        given object and the coordinate of the object in different systems

    """
    
    coordinate = [float(alpha), float(delta)]
    query  = {
              'coordinate': coordinate,
              'system': system,
              'parameter': parameter,
              'value': float(value)
             }
    headers = {'Content-type': 'application/json'}
    
    API_url = 'http://edd.ifa.hawaii.edu/CF4calculator/api.php'
    
    try:
        r = requests.get(API_url, data=json.dumps(query), headers=headers)
        output = json.loads(r.text) # a python dictionary
    except:
        print("Something went wrong!")  
        print("Please check your intput parameters ...")
        output = None

    return output

### Large lists of galaxies

- The following function, makes API calls with a large list of galaxies.
- See "example 5" to learn how to submit a list of galaxies and process the returned results.

In [2]:
import requests
import json

def CF4_list(alpha_lst, delta_lst, system='supergalactic', 
                      parameter='distance', values=[20]):
    
    """
    Inputs: 
        alpha: [array] (float) [deg]
            first coordinate parameter  (RA,  Glon, SGL)
        delta: [array] (float) [deg]
            second coordinate parameter (Dec, Glat, SGB)  
        system: (string)
            coordinate system: 
            Options are:
                "equatorial"
                "galactic"
                "supergalactic"
        parameter: (string)
            the quantity whose value is provided
            Options are:
                "distance"
                "velocity"
        value: [array] (float)
            the value of the input quantity
            distance in [Mpc] and velocity in [km/s]
        
    Output:
        A python dictionary which contains the distance and velocity of the 
        given object and the coordinate of the object in different systems

    """
    
    if len(alpha_lst)!=len(delta_lst) or len(delta_lst)!=len(values):
        return {"message": "Inconsistent sizes of the input arrays !"}
    
    payload = {}
    payload["galaxies"] = []
    
    for i in range(len(alpha_lst)):
        
        coordinate = [float(alpha_lst[i]), float(delta_lst[i])]
        galDict  = {
                  'coordinate': coordinate,
                  'system': system,
                  'parameter': parameter,
                  'value': float(values[i])
                 }
        
        payload["galaxies"].append(galDict)
    
    
    headers = {'Content-type': 'application/json'}
    
    API_url = 'http://edd.ifa.hawaii.edu/CF4calculator/api.php'
    
    try:
        r = requests.get(API_url, data=json.dumps(payload), headers=headers)
        output = json.loads(r.text) # a python dictionary
    except:
        print("Something went wrong!")  
        print("Please check your intput parameters ...")
        output = None

    return output

## Example 1

### Sending a request to the Cosmicflows-4 D-V calculator (d < 500 Mpc)
http://edd.ifa.hawaii.edu/CF4calculator

<img src="./cf4_1.png" style="width: 75%">

In [3]:
example_1 = CF4(102, -2, system='supergalactic', parameter='velocity', value=1000)

example_1

{'message': 'Success',
 'RA': 187.78917,
 'Dec': 13.33386,
 'Glon': 282.96547,
 'Glat': 75.4136,
 'SGL': 102.0,
 'SGB': -2.0,
 'observed': {'velocity': 1000.0, 'distance': [16.48, 18.14, 22.96]}}

Here, **example_1** is a Python dictionary. The keys of the output dictionary are self explanatory.

There is an additional key, **message** that holds the message of the backend code that generates the output values. In case of an unsuccessful API, *message* holds the cause of error that helps to correct the mistaken outputs.

In the following cell we show how to extract distance from the output of the calucaltor. Please *note* that the generated distance is always provided as a list, becuase multiple distances can be associated to one radial velocity.

In [4]:
distance_1 = example_1["observed"]["distance"]

distance_1

[16.48, 18.14, 22.96]

#### Example 2

<img src="./cf4_2.png" style="width: 75%">

**How to extract the radial velocity of an object with a given distance**

    SGL = 102 deg
    SGB = -2 deg
    Coordinate system = supergalactic
    input distance = 250 Mpc
    Calculator: CF4 (http://edd.ifa.hawaii.edu/CF4calculator)
    


In [5]:
example_2 = CF4(102, -2, system='supergalactic', parameter='distance', value=250)

example_2

{'message': 'Success',
 'RA': 187.78917,
 'Dec': 13.33386,
 'Glon': 282.96547,
 'Glat': 75.4136,
 'SGL': 102.0,
 'SGB': -2.0,
 'observed': {'velocity': 19190.43, 'distance': [250.0]},
 'peculiar_velocity': {'SG_Vx': -461.45, 'SG_Vy': 775.58, 'SG_Vz': -3.99}}

The peculiar velocity of a test particle in the CF4 field is stored as `peculiar_velocity`. The velocity components are presented along the three coordinate axes of the Supergalactic Cartesian system.

<a id='example3'></a>
# Example 3

### API call for a large list of galaxies

The updated version of this API is capable of processing large list of galaxies through a single call. The maximum number of galaxies per query is limited to 500.

*Hint:* You can find our example input list (**test_api.csv**) [HERE](https://raw.githubusercontent.com/ekourkchi/Cosmicflows_API/main/test_api.csv).   

### Loading the sample data set

We use `pandas` package to read data from a csv file.

In [6]:
import pandas as pd
import numpy as np

# reading the list into a pandas dataFrame
data = pd.read_csv("../test_api.csv")

# modifying the headers
for col in data.columns:
    newcol = col.split("(")[0].strip()
    data.rename(columns={col:newcol}, inplace=True)
    
data

,id,RA,Dec,Vls
0,27,6.1965,-20.7324,16092
1,76,9.8597,6.7340,12139
2,119,14.0672,-1.2561,13460
3,147,17.1579,2.2684,13163
4,151,17.2130,-15.4072,16064
5,168,18.7404,0.4311,13597
6,189,20.8597,1.7049,9720
7,193,21.2814,8.6992,14676
8,194,21.4952,-1.3394,5495
9,195,21.7281,19.2139,12841


### Function `CF4_list`

- (define above) preprocesses a list of galaxies and sends a large JSON payload that includes a possibly large list of galaxies. To avoid overloading, the maximum number of galaxies per query is limited to 500.

- Returned distances are stored in a list, because in some cases multiple distances are mapped into the same distance.

**Note:** The returned results are presented in the JSON format. In the following cell, we assume that all queried values in the list are valid and there is no error in the output results. In this case, the output JSON results could be simply transformed to a pandas dataframe.

In [7]:
N = len(data)
print("Number of objects: ", N)

## list of values
ra = data.RA.values    # deg
dec = data.Dec.values  # deg
vls = data.Vls.values  # km/s

output = CF4_list(ra, dec, system='equatorial', parameter='velocity', 
                      values=vls)
        
## output is a python dictionary, that holds the results for all indivual galaxies in a 
## list under the "results" key. 

# assuming that all input values are valid and no error messages is present in the "ouput" dictionary
results = pd.DataFrame.from_dict(output["results"])

results.head()

Number of objects:  10


,message,RA,Dec,Glon,Glat,SGL,SGB,observed
0,Success,6.19650,-20.7324,77.93223,-81.17223,274.47665,1.29298,"{'velocity': 16092.0, 'distance': [219.99]}"
1,Success,9.85969,6.7340,117.59732,-56.01341,301.90532,5.54791,"{'velocity': 12139.0, 'distance': [160.54]}"
2,Success,14.06719,-1.2561,125.69723,-64.10191,295.32585,-0.64141,"{'velocity': 13460.0, 'distance': [180.51]}"
3,Success,17.15789,2.2684,131.62935,-60.31268,299.55862,-2.65912,"{'velocity': 13163.0, 'distance': [176.03]}"
4,Success,17.21300,-15.4072,142.85777,-77.60009,282.44606,-7.34966,"{'velocity': 16064.0, 'distance': [221.26]}"


### Preparing the output table

Here, we append a few columns of the results table to the input data table. 

In [8]:
data["Glon"] = results["Glon"]
data["Glat"] = results["Glat"]
data["SGL"]  = results["SGL"]
data["SGB"]  = results["SGB"]

data["velocity_observed"] = results.apply(lambda row: row.observed["velocity"], axis=1)
data["distance_observed"] = results.apply(lambda row: row.observed["distance"], axis=1)

data.head()

,id,RA,Dec,Vls,Glon,Glat,SGL,SGB,velocity_observed,distance_observed
0,27,6.1965,-20.7324,16092,77.93223,-81.17223,274.47665,1.29298,16092.0,[219.99]
1,76,9.8597,6.7340,12139,117.59732,-56.01341,301.90532,5.54791,12139.0,[160.54]
2,119,14.0672,-1.2561,13460,125.69723,-64.10191,295.32585,-0.64141,13460.0,[180.51]
3,147,17.1579,2.2684,13163,131.62935,-60.31268,299.55862,-2.65912,13163.0,[176.03]
4,151,17.2130,-15.4072,16064,142.85777,-77.60009,282.44606,-7.34966,16064.0,[221.26]


In [9]:
# saving the resulting dataFrame that contains a distance column
data.to_csv('output_api.csv')

A copy of the output file is stored here: [output_api.csv](https://raw.githubusercontent.com/ekourkchi/Cosmicflows_API/main/output_api.csv).

# Example 4

## Using `curl` to make API requests

- Prepare a JSON payload and save it in the same directory as this notebook. We call this file `input.json`.
- Use `curl` or other favorite clients to send API requests
- In this example, coordinates of 3 galaxies have been included in the payload
- In the case of the third galaxy, Latitude is not valid as it should be between -90 and 90 degrees, which would be reflected in the corresponding record in the output JSON file 

In [10]:
%%writefile "input.json"

{
   "galaxies":[
      {
         "coordinate":[
            6.19,
            -20.73
         ],
         "system":"equatorial",
         "parameter":"distance",
         "value":10
      },
      {
         "coordinate":[
            102,
            -2
         ],
         "system":"supergalactic",
         "parameter":"velocity",
         "value":1000
      },    
      {
         "coordinate":[
            50,
            120
         ],
         "system":"supergalactic",
         "parameter":"velocity",
         "value":1000
      }   
   ]
}

Overwriting input.json


### Making an API call

In [11]:
out_file = "output.json"

! curl -d @input.json -H "Content-Type application/json"  \
    http://edd.ifa.hawaii.edu/CF4calculator/api.php > {out_file}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1075    0   550  100   525    155    148  0:00:03  0:00:03 --:--:--   303


### Loading the results

Here, `out_dict` is a python dictionary.

In [12]:
out_dict = json.load(open(out_file, "r"))

out_dict

{'results': [{'message': 'Success',
   'RA': 6.19,
   'Dec': -20.73,
   'Glon': 77.91346,
   'Glat': -81.16636,
   'SGL': 274.47712,
   'SGB': 1.29949,
   'observed': {'velocity': 989.33, 'distance': [10.0]},
   'peculiar_velocity': {'SG_Vx': -288.85, 'SG_Vy': 96.51, 'SG_Vz': -345.08}},
  {'message': 'Success',
   'RA': 187.78917,
   'Dec': 13.33386,
   'Glon': 282.96547,
   'Glat': 75.4136,
   'SGL': 102.0,
   'SGB': -2.0,
   'observed': {'velocity': 1000.0, 'distance': [16.48, 18.14, 22.96]}},
  {'message': 'Wrong Input (value out of range), Latitutde must be between -90 and 90 degrees'}]}

### How to acknowledge this work

*If you use the results of this work in your research or other applications, please cite 

- [Kourkchi et al. 2020, AJ, 159, 67](https://ui.adsabs.harvard.edu/abs/2020AJ....159...67K/abstract)
- [Valade et al. 2024, Nature Astronomy, 234 (arXiv:2409.17261)](https://ui.adsabs.harvard.edu/abs/2024NatAs.tmp..234V/abstract)